# DESBLOQUEAR PROCESOS POSTGRES


In [ ]:

"""
Script de diagnóstico y desbloqueo de procesos bloqueados en PostgreSQL
Autor: [EWE]
Descripción: Lista bloqueos activos y permite cancelar o terminar conexiones específicas.

Requiere: psycopg2
Instalar con: pip install psycopg2-binary
"""

import psycopg2 as pg2
from tabulate import tabulate
from dotenv import load_dotenv
import os

# ====================== CONFIGURACIÓN Y LOGGING ======================
load_dotenv()

# Variables de entorno
SQL_SERVER = os.getenv("SQL_SERVER")
SQL_USER = os.getenv("SQL_USER")
SQL_PASSWORD = os.getenv("SQL_PASSWORD")
SQL_DATABASE = os.getenv("SQL_DATABASE")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")
PG_DB = os.getenv("PG_DB")
PG_USER = os.getenv("PG_USER")
PG_PASSWORD = os.getenv("PG_PASSWORD")

# Configuración de conexión
def open_pg_conn():
    return pg2.connect(
        dbname=PG_DB,
        user=PG_USER,
        password=PG_PASSWORD,
        host=PG_HOST,
        port=PG_PORT
    )

def listar_bloqueos():
    query = """
    WITH lock_info AS (
        SELECT
            blocked_locks.pid       AS blocked_pid,
            blocked_activity.query  AS blocked_query,
            blocking_locks.pid      AS blocking_pid,
            blocking_activity.query AS blocking_query,
            now() - blocked_activity.query_start AS tiempo_bloqueado
        FROM pg_catalog.pg_locks blocked_locks
        JOIN pg_catalog.pg_stat_activity blocked_activity
            ON blocked_activity.pid = blocked_locks.pid
        JOIN pg_catalog.pg_locks blocking_locks
            ON blocking_locks.locktype = blocked_locks.locktype
            AND blocking_locks.database IS NOT DISTINCT FROM blocked_locks.database
            AND blocking_locks.relation IS NOT DISTINCT FROM blocked_locks.relation
            AND blocking_locks.page IS NOT DISTINCT FROM blocked_locks.page
            AND blocking_locks.tuple IS NOT DISTINCT FROM blocked_locks.tuple
            AND blocking_locks.virtualxid IS NOT DISTINCT FROM blocked_locks.virtualxid
            AND blocking_locks.transactionid IS NOT DISTINCT FROM blocked_locks.transactionid
            AND blocking_locks.classid IS NOT DISTINCT FROM blocked_locks.classid
            AND blocking_locks.objid IS NOT DISTINCT FROM blocked_locks.objid
            AND blocking_locks.objsubid IS NOT DISTINCT FROM blocked_locks.objsubid
            AND blocking_locks.pid != blocked_locks.pid
        JOIN pg_catalog.pg_stat_activity blocking_activity
            ON blocking_activity.pid = blocking_locks.pid
        WHERE NOT blocked_locks.granted
    )
    SELECT * FROM lock_info
    ORDER BY tiempo_bloqueado DESC;
    """
    with open_pg_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            rows = cur.fetchall()
            if not rows:
                print("✅ No hay procesos bloqueados.")
                return
            print(tabulate(rows, headers=[desc.name for desc in cur.description], tablefmt="fancy_grid"))

            for row in rows:
                blocked_pid = row[0]
                blocking_pid = row[2]
                print(f"⚠️ Proceso bloqueado PID={blocked_pid}, bloqueado por PID={blocking_pid}")
                decision = input(f"¿Deseás [c]ancelar o [t]erminar el PID {blocking_pid}? (c/t/n): ").lower()
                if decision == 'c':
                    cur.execute("SELECT pg_cancel_backend(%s);", (blocking_pid,))
                    print(f"⏹ Cancelado: {cur.fetchone()[0]}")
                elif decision == 't':
                    cur.execute("SELECT pg_terminate_backend(%s);", (blocking_pid,))
                    print(f"☠️ Terminado: {cur.fetchone()[0]}")
                else:
                    print("⏩ Sin acción.")

if __name__ == "__main__":
    listar_bloqueos()
